# Make CNN training set

The goal of this notebook is to provide a workflow for annotating cells in subvolumes using Neuroglancer. This notebook does a few things:
1. Starts a Neuroglancer session and provides the link
2. Loads in your cell channel data into Neuroglancer
3. Allows you to define the bounds and size of subvolumes and then draw them on Neuroglancer 
4. Allows you to create an annotation layer which will contain the annotations that you will draw and then ultimately save out. 


# Setup
In order to run the code in this notebook, you will need a conda environment with python3 and containing some additional libraries. This environment, which I call "ng" below but you could call whatever you want, can be set up in the following way:

- First download and install anaconda3. You can see if you already have it by typing "which conda" into your linux terminal. If the result shows you a filepath then you already have it installed. 

Once you have anaconda, in a linux terminal run the following commands to set up your new conda environment:
- conda create -n ng python=3.8 -y
- conda activate ng 
- pip install cloud-volume
- pip install neuroglancer <br>
- pip install --user ipykernel
- python -m ipykernel install --user --name=ng

Once this is all installed, start the jupyter notebook server  make sure to select this conda environment as the kernel when running this notebook via Kernel -> Change Kernel (you may have to restart the jupyter notebook server if you just created the conda environment)

## Host your brain data via cloudvolume

A quirk with Neuroglancer is that the data need to be hosted via a server in order for Neuroglancer to load them. You will use your own computer as a server for your data. Below is code that will host your cell channel 642 data for the brain zimmerman_02-f12 on port 1337 of your machine. Note that you must run the following code in a separate python session (make sure to activate the "ng" conda environment before starting the python session). If you just ran it in this jupyter notebook it would work but it would cause the notebook to hang and you wouldn't be able to run the rest of the code. Note that you will need to have the LightSheetData bucket mounted on whatever machine you're working on for this to work. 
```python
import cloudvolume
vol = cloudvolume.CloudVolume('file:///jukebox/LightSheetData/lightserv/cz15/zimmerman_02/zimmerman_02-f12/imaging_request_1/viz/rawdata/channel_642_corrected')
vol.viewer(port=1337)  
```
Later when you want to host the rest of the brains, the only part of the filepath above that you need to change is the sample name, i.e. zimmerman_02-f12.

## Imports

In [1]:
import numpy as np
import neuroglancer as ng

In [2]:
# Use the BRAIN CoGS client
ng.set_static_content_source(url='https://neuroglancer-braincogs.appspot.com')

## Load the data you are hosting on port 1337 into Neuroglancer and generate the link
First set the sample name. In this example we are using:

In [3]:
sample_name = "zimmerman_02-f11"

In [4]:
viewer = ng.Viewer()
with viewer.txn() as s:
    s.layers[f'{sample_name} cell channel'] = ng.ImageLayer(source='precomputed://http://localhost:1337')
print(viewer)

http://127.0.0.1:38145/v/e8c0d885551b7dfed36f1b63be72be3e6255be5a/


Click the link above to open Neuroglancer in a new tab. I recommend using Google Chrome for your Neuroglancer sessions. If you don't open the link first none of the code below will work. If you re-run the cell above it will generate an entirely new Neuroglancer session and a new link.

You may notice that the background is quite dark. Use the "f" key to increase brightness and "d" key to decrease brightness. "i" inverts the colors. The "h" key brings up the help menu with the whole list of commands.

## Make a subvolume box for annotations
To define a subvolume, you need to supply the size and the location of the subvolume. 

First, we will define the size in pixels, keeping in mind that 200x200x50 is a minimum
The pixel size is 1.8 um x 1.8 um x 2 um (x,y,z)

In [5]:
npix_subvol = [300,300,50] # x,y,z 

Now decide where to put the subvolume. Navigate in Neuroglancer to the place where you want the subvolume to start, then right click that spot in any of the 2D viewer panels. The x,y,z coordinates of where you clicked (also the location of the red, green and blue axis lines) will be shown in white in the top left of the screen. Update the list below with those coordinates. Note that these coordinates will be the origin (not center) of the subvolume. 

In [6]:
voxel_offset_origvol = [3200,3500,1800] # x,y,z of where the subvolume starts. 

Now run the cell below without changing anything. Then look in Neuroglancer and your subvolume will be created as a red box. 
The screen will automatically pan to somewhere near the origin of the subvolume. 

In [8]:
def make_subvolume(npix_subvol,voxel_offset_origvol):  
    use_mm_offset=False
    xzfill,yzfill,zzfill = [str(coord).zfill(4) for coord in voxel_offset_origvol]
    layer_name = f'{sample_name}_subvol_x{xzfill}_y_{yzfill}_z{zzfill}_dim_cells'
    with viewer.txn() as s:
        dim_dict = s.dimensions
        nm_scale_dict = {}
        for coord in ['x','y','z']:
            dim_obj = dim_dict[coord]
            unit = dim_obj.unit
            assert unit == 'm'
            nm_scale_dict[coord] = dim_obj.scale*1e9
    nm_scale_dict

    if not use_mm_offset:
        mm_offset_x = voxel_offset_origvol[0]*nm_scale_dict['x']*1e-6 
        mm_offset_y = voxel_offset_origvol[1]*nm_scale_dict['y']*1e-6 
        mm_offset_z = voxel_offset_origvol[2]*nm_scale_dict['z']*1e-6 
        mm_offset = [mm_offset_x,mm_offset_y,mm_offset_z]

    voxel_offset = [int(round(mm_offset[0]*1e6/nm_scale_dict['x'])),
                    int(round(mm_offset[1]*1e6/nm_scale_dict['y'])),
                    int(round(mm_offset[2]*1e6/nm_scale_dict['z']))]
    subvol = np.zeros(npix_subvol, dtype=np.uint8)

    with viewer.txn() as s:
        s.layers[layer_name] = ng.ImageLayer(
            source=ng.LocalVolume(
                subvol,
                dimensions=ng.CoordinateSpace(
                    scales=[nm_scale_dict['x'], nm_scale_dict['y'], nm_scale_dict['z']], 
                    units=['nm', 'nm', 'nm'],
                    names=['x', 'y', 'z']),
                voxel_offset=voxel_offset), 
        )
        # Then pan to the location of this new volume
        # To do that, need dimensions of your original data layer

        # find the position of the new subvolume in these units
        x_subvol_orig = mm_offset[0]*1e6/nm_scale_dict['x']+10
        y_subvol_orig = mm_offset[1]*1e6/nm_scale_dict['y']+10
        z_subvol_orig = mm_offset[2]*1e6/nm_scale_dict['z']
        layer = s.layers[layer_name]
        layer.annotationColor = "#ff0000"
        s.selected_layer.layer = layer_name 
        s.selected_layer.visible = True
        # Pan/zoom to near the origin of the subvolume 
        s.voxel_coordinates = [x_subvol_orig,y_subvol_orig,z_subvol_orig]
        s.cross_section_scale=0.5

make_subvolume(npix_subvol,voxel_offset_origvol)

You should see a red square show up in at least one of the 2D panels and in the 3d viewer panel. 

It might be a little darker or lighter than the rest of the volume. Use the opacity slider in the "Rendering" tab on the right hand side panel which should also have popped up to change the contrast between the subvolume and the rest of the volume. ctrl+mousewheel zooms in and out (or two finger drag up/down on a laptop).  

In [10]:
# Finally, make an annotation layer so you can start making annotations
with viewer.txn() as s:
    xzfill,yzfill,zzfill = [str(coord).zfill(4) for coord in voxel_offset_origvol]
    layer_name = f'{sample_name}_annotation_x{xzfill}_y_{yzfill}_z{zzfill}_dim_cells'
    s.layers[layer_name]=ng.AnnotationLayer()
    annot_layer = s.layers[layer_name]
    annot_layer.tool = "annotatePoint"
    s.cross_section_depth=-40 # conrols the z-depth in microns over which the annotated points will persist. 
    # negative sign is necessary for some reason.
    # Toggle this in Neuroglancer with alt+"=" or alt+"-" to decrease and increase the depth
    s.selected_layer.layer = layer_name 
    s.selected_layer.visible = True
with viewer.config_state.txn() as st:
    st.status_messages['update'] = 'Annotate points by ctrl+left clicking'   
    

Annotate points by ctrl+left clicking. You can control the size of the points via the size slider in the "Rendering" tab on the right side control panel. When you are done with your annotations, use the "Export to CSV" button on the right hand side panel under the "Annotations" tab. Just make sure that the correct annotation layer is selected (right click on the layer box at the top of the screen and it will be outlined in green) and save the file with the filename below:

In [ ]:
xstart = str(voxel_offset_origvol[0]).zfill(4)
ystart = str(voxel_offset_origvol[1]).zfill(4)
zstart = str(voxel_offset_origvol[2]).zfill(4)
annotation_savename = f"{sample_name}_x{xstart}_y{ystart}_z{zstart}_annotations.csv"
print(annotation_savename)